In [66]:
import pandas as pd
from datetime import datetime

In [93]:
df = pd.read_csv('/Users/kaitlinsinger/wa_notify/DEN_future/data_extraction/manipulated_data/simplified_wide_df_with_date_loc.csv')

In [94]:
#[x for x in df.columns if x.endswith('count') or x.endswith('perc')]
df.columns

Index(['Unnamed: 0', 'source', 'pm_start_date', 'pm_end_date', 'pm_location',
       'cases_assigned_count', 'cases_interviewed_count',
       'cases_interviewed_perc', 'cases_named_contacts_count',
       'cases_not_assigned_count', 'cases_per_staff_month',
       'cases_positive_from_test_mean', 'cases_positive_from_test_med',
       'cases_reached_count', 'cases_reached_from_exposure_mean',
       'cases_reached_from_positive_max', 'cases_reached_from_positive_med',
       'cases_reached_from_symptom_mean', 'cases_reached_from_symptom_med',
       'cases_reached_from_test_mean', 'cases_reached_from_test_med',
       'cases_reached_perc', 'cases_were_contacts_count',
       'contacts_assigned_count', 'contacts_infected_count',
       'contacts_infected_perc', 'contacts_interviewed_count',
       'contacts_interviewed_perc', 'contacts_named_count',
       'contacts_named_count_household', 'contacts_named_perc_household',
       'contacts_named_ratio_mean', 'contacts_named_ratio_mean_c

In [95]:
# will need to fix these rows manually
df[df['pm_location'].str.endswith('Connecticut')]['pm_location'] = 'Connecticut'

/var/folders/5_/xbdfsmkd6pq54320tdmh8fth0000gn/T/ipykernel_49840/2889793188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['pm_location'].str.endswith('Connecticut')]['pm_location'] = 'Connecticut'


In [99]:
# list of cols that have both count and perc
cols_to_impute = ['cases_assigned', 'cases_interviewed', 'cases_reached',
                  'contacts_infected', 'contacts_interviewed', 'contacts_named',
                 'contacts_reached']

# TODO: ratio of reported contacts to contacts reporting cases
# add perc naming contacts

imputation_dict = {
    #'cases_assigned_count': '?', 
    #'cases_assigned_perc': 'df["cases_assigned_count"]/(df["cases_assigned_count"]+df["cases_not_assigned_count"])*100', 
    #'cases_interviewed_count': '?', 
    'cases_interviewed_perc': 'df["cases_interviewed_count"]/df["cases_assigned_count"]*100',
    #'cases_reached_count': '',
    'cases_reached_perc': 'df["cases_reached_count"]/df["cases_assigned_count"]*100',
    
    #'contacts_named_count': '',
    # take another look at this
    # percent of cases who named at least one contact - rerun but see fill of orig data
    'percent_naming_contacts': 'df["cases_named_contacts_count"]/df["cases_interviewed_count"]*100',
    #'contacts_infected_count': '', 
    # contacts_named_count or contacts_interviewed_count (which is more populated) - look at paper's definition
    'contacts_infected_perc': 'df["contacts_infected_count"]/df["contacts_named_count"]*100',
    #'contacts_interviewed_count': '', 
    'contacts_interviewed_perc': 'df["contacts_interviewed_count"]/df["contacts_named_count"]*100',
    #'contacts_reached_count': '',
    'contacts_reached_perc': 'df["contacts_reached_count"]/df["contacts_named_count"]*100'
}

In [100]:
# only inputes is the data is missing
for column, equation in imputation_dict.items():
    df[column] = df.apply(
        lambda row: eval(equation, {}, {"df": row}) if pd.isna(row[column]) else row[column], 
        axis=1
    )

In [101]:
df.head()

,Unnamed: 0,source,pm_start_date,pm_end_date,pm_location,cases_assigned_count,cases_interviewed_count,cases_interviewed_perc,cases_named_contacts_count,cases_not_assigned_count,...,num_users_upper_bound,percent_naming_contacts,quarantine_adherent_perc,staff_hired_ci_count,staff_hired_ci_per_health_dept_mean,staff_hired_count,staff_hired_ct_count,staff_hired_ct_per_health_dept_mean,user_cases_notifying,users_with_code
0,0,A Multifaceted Evaluation of a COVID-19 Contac...,3/1/21,3/31/21,King County,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A Multifaceted Evaluation of a COVID-19 Contac...,4/1/21,6/30/20,King County,NaN,NaN,76.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A Multifaceted Evaluation of a COVID-19 Contac...,7/1/20,6/30/21,King County,52317.0,42900.0,82.000115,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,A Multifaceted Evaluation of a COVID-19 Contac...,7/1/20,9/30/20,King County,NaN,NaN,87.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,A Patient-Initiated Digital COVID-19 Contact N...,5/18/20,8/17/20,United States*,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
region = {
    "loc_A": "West",
    "loc_B": "South",
    "loc_C": "Northeast",
    "loc_D": "Midwest",
    "loc_E": "West",
    "loc_F": "Midwest",
    "loc_G": "West",
    "loc_H": "Midwest",
    "loc_I": "South",
    "loc_J": "West",
    "loc_K": "Northeast",
    "loc_L": "South",
    "loc_M": "Midwest",
    "loc_N": "South",
    "King County": "Northwest",
    "WA": "Northwest",
    "Washington": "Northwest",
    "central WA": "Northwest",
    "Maryland": "Northeast",
    "Kansas": "Midwest",
    "United States": "US",
    "across US": "US",
    "United States*": "US",
    "Philadelphia": "Midwest",
    "Maimi University": "Midwest", #OH
    "San Fransico": "Northwest", #typo?
    'San Francisco': "Northwest",
    "California": "Northwest",
    "Members of large CA health care plan": "Northwest",
    "Boston University": "Northeast",
    "Oak Ridge National Laboratory": "South", #TN
    "New York City": "Northeast",
    "NY": "Northeast",
    "New York": "Northeast",
    "New York City communities with higher than average COVID-19 incidence rates": "Northeast",
    "New York except NYC": "Northeast",
    "North Carolina": "Northeast",
    "Randolph County": "Northeast", #NC
    "Mecklenburg": "Northeast", #NC
    "64 HD accross US and collonies": "US",
    "Connecticut": "Northeast",
    '\xa0Connecticut': "Northeast",
    "New Haven": "Northeast",
    "Michigan": "Midwest",
    "Salt Lake City": "Northwest",
    "DE": "Northeast",
    "Fulton County": "South", #ATL
    "VT": "Northeast",
    "Chicago": "Midwest",
    "Maricopa": "South", #AZ
    "Alaska": "Alaska"
}

In [103]:
# https://www.cdc.gov/ncird/whats-new/covid-19-can-surge-throughout-the-year.html
time_periods = {
    "1/1/2020-5/1/2020": "T1",
    "5/2/2020-10/15/2020": "T2",
    "10/16/2020-3/1/2021": "T3",
    "3/2/2021-5/15/2021": "T4",
    "5/15/2021-9/15/2021": "T5",
    "9/16/2021-3/1/2022": "T6",
    "3/2/2022-9/15/2022": "T7",
    "9/16/2022-3/15/2023": "T8",
    "3/16/2023-9/1/2023": "T9",
    "9/2/2023-3/15/2024": "T8"
}

def assign_time_period(date):
    for start, end, period in time_ranges:
        if start <= date <= end:
            return period
    return None

time_ranges = []
for k, v in time_periods.items():
    start_str, end_str = k.split("-")
    start_dt = datetime.strptime(start_str, "%m/%d/%Y")
    end_dt = datetime.strptime(end_str, "%m/%d/%Y")
    time_ranges.append((start_dt, end_dt, v))

In [104]:
# replace pm_location with region
df['region'] = df['pm_location'].map(region)

# add time period data
df["pm_start_date"] = pd.to_datetime(df["pm_start_date"])
df["time_period"] = df["pm_start_date"].apply(assign_time_period)

In [105]:
df[df.region.isna()]

,Unnamed: 0,source,pm_start_date,pm_end_date,pm_location,cases_assigned_count,cases_interviewed_count,cases_interviewed_perc,cases_named_contacts_count,cases_not_assigned_count,...,quarantine_adherent_perc,staff_hired_ci_count,staff_hired_ci_per_health_dept_mean,staff_hired_count,staff_hired_ct_count,staff_hired_ct_per_health_dept_mean,user_cases_notifying,users_with_code,region,time_period
65,65,Digital Point Solutions for Extending Contact ...,2021-03-15,4/13/21,state_a,21540.0,NaN,NaN,NaN,74869.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T4
72,72,NPR,2021-12-01,1/1/22,Alabama,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,NaN,T6
74,74,NPR,2021-12-01,1/1/22,American Samoa,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,T6
75,75,NPR,2021-12-01,1/1/22,Arizona,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,396.0,NaN,NaN,NaN,NaN,T6
76,76,NPR,2021-12-01,1/1/22,Arkansas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,840.0,NaN,NaN,NaN,NaN,T6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,211,NPR,2021-09-01,10/1/21,Vermont,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN,NaN,T5
212,212,NPR,2021-09-01,10/1/21,Virginia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2503.0,NaN,NaN,NaN,NaN,T5
214,214,NPR,2021-09-01,10/1/21,West Virginia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,350.0,NaN,NaN,NaN,NaN,T5
215,215,NPR,2021-09-01,10/1/21,Wisconsin,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,175.0,NaN,NaN,NaN,NaN,T5


In [106]:
df['ratio_cases_contacts_intv_computed'] = df['cases_interviewed_count']/df['contacts_interviewed_count']
df['contacts_named_ratio_computed'] = df['contacts_named_count']/df['cases_interviewed_count']
df['contacts_named_naming_ratio_computed'] = df['contacts_named_count']/df['cases_named_contacts_count']

In [107]:
df.round(2).to_csv('/Users/kaitlinsinger/wa_notify/DEN_future/data_extraction/manipulated_data/simplified_wide_df_with_date_loc_imputed.csv')